In [18]:
import pandas as pd

# df = pd.read_csv('data_extracted_transformed_openai.csv')

df = pd.read_csv('llm_test_data.csv')

In [19]:
columns_to_check = ['age','vehicle_type', 'reason', 'fatalities', 'injured', 'gender', 'time']
print('*******Unique values*******')
for columns in columns_to_check:
    print('_'*10,'\n')
    print(df[columns].unique())

*******Unique values*******
__________ 

['55,22' '32' '50,19' '30,26,27' '17-year-old, 42-year-old' nan
 '57,55,27,22,20,25' '38' '53,35,31' '25' '16,13,18' '29,23,25' '42,23,48'
 '34' '71' '80,54,51' '40,11,7' '26, 32' '50, 42' '1.5' '22,24,20'
 '42,12,6' '35' '35,29,20,31' '2' '38,36,11,7' '34, 67'
 '35,9,55, 25, 42,45,14,25,14,9' '25, 43,52' '32,22,20,19'
 '26,2,9 months, 6 months' '35,38,14,9,4' '28,10,67']
__________ 

['bike, car' 'bike,Dumper(truck),car' 'bike, truck, bus' 'car, truck'
 'bike,truck, car, concrete mixing vehicle' 'pickup vehicle, car' 'car'
 'car, truck,bus' 'bike,car' 'car,trucks' 'car,truck' 'autorickshaw, bike'
 'car,bike' 'truck,trailer,car' 'motorcycle,car,truck' 'truck,car'
 'car,motorcycle' 'car,minibus' 'taxi,sedan' 'car,lorry'
 'four-wheeler,car' 'truck, car' 'car, JCB' 'two-wheeler,bus,car'
 'van,bus' 'two-wheelers, autorickshaw' 'canter truck,car'
 'car, container truck' 'cargo truck, motorcycle, car' 'car,two-wheeler'
 'car, jeep']
__________ 

[nan 

import os
import json
import time
import numpy as np
import pandas as pd
import ast
import re
from datetime import datetime
from word2number import w2n
from tqdm import tqdm# __init__

In [28]:
import os
import json
import time
import numpy as np
import pandas as pd
import ast
import re
from datetime import datetime
from word2number import w2n
from tqdm import tqdm

# Functions

In [29]:
# Function to handle number of injured and fatalities when there's only a single entry
def text_to_single_number(text):
    try:
        # Parse the text to extract numbers
        numbers = [w2n.word_to_num(item.strip()) if '.' not in item else float(item.strip()) 
                   for item in text.strip('[]').split(',') if item.strip()]
        # Return the number if there's exactly one, or np.nan otherwise
        return numbers[0] if len(numbers) == 1 else np.nan
    except ValueError:
        return np.nan

In [30]:
#Function to handle reason of accident
reason_mappings = {
    'Speed': ['speeding', 'speed', 'racing', 'joy'],
    'Driving Under Influence': ['drunk', 'drunken', 'alcohol', 'drink', 'drug'],
    'Collision': ['hit-and-run', 'collided', 'collision', 'mowing', 'mow', 'rammed','hit','accident'],
    'Visibility': ['fog', 'fogged', 'smog', 'light','rain'],
    'Negligence': ['negligence','skid','wrong side','negligent', 'rash', 'reckless', 'dozed','abrupt','control','rear-ended','sleep','crossing','jaywalking'],
    'External influences': ['tree','cattle','divider', 'rock', 'stones', 'brick','electric', 'electricity', 'dementia', 'tyre', 'overloaded']
}
# Function to handle reason of accident with hierarchy
def categorize_accident_reason(reason_str):
    if pd.isnull(reason_str):
        return np.nan

    reason_str = reason_str.lower()
    categories = set()
    
    # Extract reason terms from the string, handling lists
    reason_list = re.findall(r'[\w-]+', reason_str.replace('[', '').replace(']', ''))
    
    # Reason hierarchy dictionary (lower number means higher priority)
    reason_hierarchy = {
        'Driving Under Influence': 1,
        'Speed': 2,
        'Negligence': 3,
        'Collision': 4,
        'Visibility': 5,
        'External influences': 6
    }
    
    # Identify categories for each reason term
    min_priority = float('inf')
    selected_category = np.nan
    
    for reason in reason_list:
        for category, keywords in reason_mappings.items():
            if any(keyword in reason for keyword in keywords):
                current_priority = reason_hierarchy.get(category, float('inf'))
                # Check if the current category has higher priority
                if current_priority < min_priority:
                    min_priority = current_priority
                    selected_category = category
    return selected_category


# # Reapply the function to the 'REASON' column
# predictions['REASON'] = predictions['REASON'].apply(categorize_accident_reason)

In [31]:
#Function to handle gender
male_keywords = {'man', 'men', 'boy', 'boys', 'son', 'sons', 'he', 'his', 'him', 'brother', 'brothers', 'father', 'grandfather', 'grandson', 'husband', 'businessman', 'businessmen', 'policeman', 'policemen', 'cameraman', 'male', 'nephew', 'uncle'}
female_keywords = {'woman', 'women', 'girl', 'girls', 'daughter', 'daughters', 'she', 'her', 'sister', 'sisters', 'mother', 'grandmother', 'granddaughter', 'wife', 'businesswoman', 'policewoman', 'female', 'niece', 'aunt', 'lady'}
def classify_gender(gender_str):
    gender_str = re.sub(r'[\[\]\']', '', gender_str)
    
    
    if pd.isnull(gender_str):
        return np.nan

    tokens = set(re.split(r'[,\s\[\]]+', gender_str.lower()))
    identified_genders = []
    if any(token in male_keywords for token in tokens):
        identified_genders.append('Male')
    if any(token in female_keywords for token in tokens):
        identified_genders.append('Female')
    return identified_genders[0] if len(identified_genders) == 1 else identified_genders if identified_genders else np.nan

In [32]:
# def extract_numbers(input_string):
#     numbers = re.findall(r'\d+', input_string)
#     numbers = [int(num) for num in numbers]
#     if len(numbers) == 0:
#         return np.nan
#     else:
#         return numbers
    
    
def extract_numbers(input_string):
    numbers = re.findall(r'\d+', input_string)
    numbers = [int(num) if int(num) != 0 else 1 for num in numbers]
    numbers = [num for num in numbers if num <= 100]
    if len(numbers) == 0:
        return np.nan
    else:
        return numbers

# # Given string
# given_string = "[('Chinta Devi', 51), ('Ram Chandra Gupta', 55), ('Maya Devi', 52), ('Vikas', 30)]"

# # Extract numbers from the given string
# extracted_numbers = extract_numbers(given_string)

# print(extracted_numbers)

In [33]:
import re

def extract_time(input_data):
    # Regular expression pattern to match morning, afternoon, evening, or night
    time_pattern = r'(morning|afternoon|evening|night)'
    
    # If input_data is a list
    if isinstance(input_data, list):
        # Iterate through the list
        for item in input_data:
            # If the item is not 'NA', try to find a match
            if item != 'NA':
                match = re.search(time_pattern, item)
                # If a match is found, return it
                if match:
                    return match.group()
    # If input_data is a string
    elif isinstance(input_data, str):
        # Find all matches in the input string
        matches = re.findall(time_pattern, input_data)
        # If matches are found, return the first match, else return None
        if matches:
            return matches[0]
    
    # If no match is found, return None
    return None

In [34]:
def extract_unique_vehicles(input_string):
    allowed_words = ['Two-Wheeler', 'Special Vehicle', 'Commercial Vehicle', 'Car']
    # Remove brackets and single quotes, split the string into words
    words = input_string.strip("[]").replace("'", "").split(", ")
    # Extract unique words
    unique_words = list(set(words))
    # Filter out words not in the allowed list
    filtered_words = [word for word in unique_words if word in allowed_words]
    
    if len(filtered_words) == 0:
        return np.nan
    else:
        return filtered_words

In [35]:
def extract_unique_reasons(input_string):
    allowed_words = ['Negligence', 'Collision', 'External influences', 'Speed', 'Driving Under Influence']
    # Remove brackets and single quotes, split the string into words
    words = input_string.strip("[]").replace("'", "").split(", ")
    # Extract unique words
    unique_words = list(set(words))
    # Filter out words not in the allowed list
    filtered_words = [word for word in unique_words if word in allowed_words]
    
    if len(filtered_words) == 0:
        return np.nan
    else:
        return filtered_words

# Preprocess

In [40]:
df['fatalities'] = df['fatalities'].astype(str).apply(text_to_single_number)
df['injured'] = df['injured'].astype(str).apply(text_to_single_number)
df['gender'] = df['gender'].astype(str).apply(classify_gender)
df['age'] = df['age'].astype(str).apply(extract_numbers)
df['time'] = df['time'].astype(str).apply(extract_time)
df['vehicle_type'] = df['vehicle_type'].astype(str).apply(extract_unique_vehicles)
df['reason'] = df['reason'].astype(str).apply(categorize_accident_reason)

In [42]:
df.to_csv('llm_test_data_t.csv')

In [44]:
df = pd.read_csv('llm_test_data_t.csv')

In [45]:
columns_to_check = ['age','vehicle_type', 'reason', 'fatalities', 'injured', 'gender', 'time']
print('*******Unique values*******')
for columns in columns_to_check:
    print('_'*10,'\n')
    print(df[columns].unique())

*******Unique values*******
__________ 

['[55, 22]' '[32]' '[50, 19]' '[30, 26, 27]' '[17, 42]' nan
 '[57, 55, 27, 22, 20, 25]' '[38]' '[53, 35, 31]' '[25]' '[16, 13, 18]'
 '[29, 23, 25]' '[42, 23, 48]' '[34]' '[71]' '[80, 54, 51]' '[40, 11, 7]'
 '[26, 32]' '[50, 42]' '[1, 5]' '[22, 24, 20]' '[42, 12, 6]' '[35]'
 '[35, 29, 20, 31]' '[2]' '[38, 36, 11, 7]' '[34, 67]'
 '[35, 9, 55, 25, 42, 45, 14, 25, 14, 9]' '[25, 43, 52]'
 '[32, 22, 20, 19]' '[26, 2, 9, 6]' '[35, 38, 14, 9, 4]' '[28, 10, 67]']
__________ 

[nan]
__________ 

[nan 'Negligence' 'Speed' 'Collision']
__________ 

[ 2.  3. nan  4.  1.  5.]
__________ 

[nan  3.  2.  4.  1.  8.]
__________ 

["['Male', 'Female']" 'Male' 'Female' nan]
__________ 

[nan 'morning' 'evening' 'night' 'afternoon']
